### ContextualCompressionRetriever

In [6]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_core.documents import Document

In [2]:
# Recreate the document objects from the previous data
docs = [
    Document(page_content=(
        """The Grand Canyon is one of the most visited natural wonders in the world.
        Photosynthesis is the process by which green plants convert sunlight into energy.
        Millions of tourists travel to see it every year. The rocks date back millions of years."""
    ), metadata={"source": "Doc1"}),

    Document(page_content=(
        """In medieval Europe, castles were built primarily for defense.
        The chlorophyll in plant cells captures sunlight during photosynthesis.
        Knights wore armor made of metal. Siege weapons were often used to breach castle walls."""
    ), metadata={"source": "Doc2"}),

    Document(page_content=(
        """Basketball was invented by Dr. James Naismith in the late 19th century.
        It was originally played with a soccer ball and peach baskets. NBA is now a global league."""
    ), metadata={"source": "Doc3"}),

    Document(page_content=(
        """The history of cinema began in the late 1800s. Silent films were the earliest form.
        Thomas Edison was among the pioneers. Photosynthesis does not occur in animal cells.
        Modern filmmaking involves complex CGI and sound design."""
    ), metadata={"source": "Doc4"})
]

In [7]:
# Create a FAISS vector store from the documents
embedding_model = HuggingFaceEmbeddings(
    model_name = "intfloat/e5-small",
    encode_kwargs = {'normalize_embeddings':True}
)

C:\Users\akash\AppData\Local\Temp\ipykernel_8896\3103384267.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
e:\15_CampusX_LangChain\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
# Store in FAISS 
vectorstore = FAISS.from_documents(docs,embedding_model)

In [11]:
base_retriever = vectorstore.as_retriever(search_kwargs={'k':5})

In [9]:
# Setup compression using LLM
llm = ChatGroq(
    model_name = "llama3-8b-8192",
    temperature = 0.5,
    streaming= False
)

# Compressor 
compressor = LLMChainExtractor.from_llm(llm)

In [13]:
# Create the Contextual Compression Retriever
compression_retriever = ContextualCompressionRetriever(
    base_retriever = base_retriever,
    base_compressor = compressor
)

In [15]:
# Query the retriever
query = "What is photosynthesis?"
compressed_results = compression_retriever.invoke(query)

In [16]:
for i,doc in enumerate(compressed_results):
    print(f"\n---- Result {i+1} ----")
    print(doc.page_content)


---- Result 1 ----
Extracted relevant parts:
Photosynthesis is the process by which green plants convert sunlight into energy.

---- Result 2 ----
The extracted relevant part is:

The chlorophyll in plant cells captures sunlight during photosynthesis.
